In [ ]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with LangChain

This notebook demonstrates how to connect to a remote **MCP Server** using LangChain's **MultiServerMCPClient**.

- Use the **Model Context Protocol (MCP)** to modularize and scale tool usage across distributed systems.
- Define and call tools hosted remotely, enabling seamless agent interaction and task execution.
- Showcase how MCP enables flexible, service-oriented tool integration for multi-agent workflows.

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.


In [1]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from utils import pretty_print_response
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent


load_dotenv(override=True)

True

In [2]:
llm = AzureChatOpenAI(
        temperature=0,
        azure_deployment=environ["AZURE_OPENAI_MODEL"],
        api_version=environ["AZURE_OPENAI_API_VERSION"]
    )

## Connecting to a remote MCP server via sse

- Server-Sent Events (SSE) is a mechanism for establishing a continuous, one-way data stream from the server to a client, such as a web application. This allows the server to push updates or messages to the client without the client needing to constantly poll for new information

In [3]:
async with MultiServerMCPClient(
{
    "weather": {
        "url": f"{environ["MCP_SERVER_URL"]}/weather/sse",
        "transport": "sse"
    },
    "data center energy usage": {
        "url": f"{environ["MCP_SERVER_URL"]}/energy/sse",
        "transport": "sse"
    },
    "document search": {
        "url": f"{environ["MCP_SERVER_URL"]}/search/sse",
        "transport": "sse"
    }
}) as client:
    agent = create_react_agent(llm, client.get_tools())
    reponse = await agent.ainvoke({"messages": "what is the weather in sf"})
    pretty_print_response(reponse)
    reponse = await agent.ainvoke({"messages": "What data centers are in 'critical'?"})
    pretty_print_response(reponse)
    reponse = await agent.ainvoke({"messages": "Describe the resource intensity of data center facility infrastructure"})
    pretty_print_response(reponse)


Human: what is the weather in sf


AI: (Tool Call Triggered)
   Calling `get_weather_tool` with args: {"location":"San Francisco"}


Tool `get_weather_tool` Response: It's 60 degrees and foggy.


AI: The weather in San Francisco is currently 60°F and foggy.


Human: What data centers are in 'critical'?


AI: (Tool Call Triggered)
   Calling `get_energy_usage_usage_energy_usage_get` with args: {"alarm_status":"critical"}


Tool `get_energy_usage_usage_energy_usage_get` Response: [{'timestamp': '2025-04-14T15:00:13.724337', 'data_center_id': 'DC-NYC1', 'zone': 'B2', 'power_draw_kw': 190.82, 'it_load_kw': 148.05, 'cooling_load_kw': 35.62, 'pue': 1.29, 'temperature_c': 29.66, 'humidity_percent': 55.6, 'ups_load_percent': 78.7, 'battery_backup_status': 'charging', 'grid_energy_source': 'solar', 'co2_emissions_kg': 108.75, 'alarm_status': 'critical', 'operator_notes': ''}, {'timestamp': '2025-04-14T14:00:13.724337', 'data_center_id': 'DC-CHI2', 'zone': 'A1', 'power_draw_kw': 162.3, 'it_load_